In [ ]:
import sys
import os

import geopandas as gpd

sys.path.append(r"C:\WBG\Work\Code\GOSTrocks\src")

%load_ext autoreload
%autoreload 2

In [ ]:
# Define source data
FUA_file = r"C:\WBG\Work\data\URBAN\GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg"
OE_file = r"C:\WBG\Work\data\URBAN\OE_FUA_SHAPEFILE\OE_FUA_SHAPEFILE.shp"

out_folder = r"C:\WBG\Work\MENA_Urban\RESULTS\EXTENT_JOIN"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
inFUA = gpd.read_file(FUA_file)
inOE = gpd.read_file(OE_file)

inOE = inOE.to_crs(inFUA.crs)

In [ ]:
inFUA.head()

In [ ]:
inOE.shape

In [ ]:
# convert inFUA geometry to centroid
inFUA["geometry"] = inFUA.centroid

# Spatial join inOE (centroids) to inFUA
joined_extents = gpd.sjoin_nearest(
    inOE.loc[:, ["OE_FUAID", "geometry"]],
    inFUA.loc[:, ["eFUA_ID", "geometry"]],
    how="inner",
    distance_col="dist_to_fua",
)
# pd.DataFrame(joined_extents.drop(['geometry', 'index_right'], axis=1)).to_csv(os.path.join(out_folder, 'OE_FUA_join_raw.csv'))
joined_extents.sort_values(by="dist_to_fua", ascending=False)

In [ ]:
joined_res = (
    joined_extents.groupby("OE_FUAID")
    .agg(count=("eFUA_ID", "count"), FUA_IDs=("eFUA_ID", "unique"))
    .reset_index()
    .sort_values("count", ascending=False)
)
joined_res.to_csv(os.path.join(out_folder, "OE_GHSL_aggregated.csv"), index=False)